In [1]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

SAVED_MODEL_PATH = 'models/qwen2-lora-distilled-final'

student_model = PeftModel.from_pretrained(
    AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct", torch_dtype=torch.float16),
    SAVED_MODEL_PATH
)
student_tokenizer = AutoTokenizer.from_pretrained(SAVED_MODEL_PATH, trust_remote_code=True)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [3]:
from torch.nn import functional

def chat_with_model(prompt, return_confidence=False):
    device = student_model.device
    inputs = student_tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    input_length = inputs["input_ids"].shape[1]

    with torch.no_grad():
        outputs = student_model.generate(
            **inputs,
            max_new_tokens=128,
            return_dict_in_generate=True,
            output_scores=True
        )

    response_ids = outputs.sequences[0][input_length:]
    response_text = student_tokenizer.decode(response_ids, skip_special_tokens=True)

    if return_confidence:
        scores = torch.stack(outputs.scores, dim=0).to(outputs.sequences.device)
        probs = functional.softmax(scores, dim=-1)
        max_probs = probs.max(dim=-1).values
        avg_confidence = max_probs.mean().item()
        return response_text, avg_confidence

    return response_text

In [4]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('sample/merged_distilled_dataset.csv')
distilled_dataset = Dataset.from_pandas(df)
distilled_dataset

Dataset({
    features: ['prompt', 'response'],
    num_rows: 10000
})

In [5]:
distilled_dataset[100]['prompt']

"User: I'm new to ubuntu, how can do I make hidden files visible?\nAssistant:"

In [6]:
sample = distilled_dataset[100]
response, confidence = chat_with_model(sample['prompt'], return_confidence=True)
print("Prompt:", sample['prompt'])
print("Response:", response)
print("Confidence:", round(confidence * 100, 2), "%")

Prompt: User: I'm new to ubuntu, how can do I make hidden files visible?
Assistant:
Response:  To make hidden files in Ubuntu accessible from the command line, you can use the `find` and `ls -l` commands. Here's a step-by-step guide:

1. Open a terminal.

2. Navigate to the directory where your file is located using `cd`.

3. Use the following command to list all files and directories in the current directory:
```
ls
```

4. Use the `-a` option followed by the name of the file or directory you want to view, like `ls /home/user/documents/`. The `-a` option lists all files and directories, not just their names.

5.
Confidence: 82.76 %
